# NY Motor Vehicle Collisions – Exploratory Data Analysis  
In this notebook, we analyze New York’s Vehicular crash data available at https://data.cityofnewyork.us/Public-Safety/Motor-Vehicle-Collisions-Crashes/h9gi-nx95


The data contains information from all police reported motor vehicle collisions in NYC. The information for this dataset is collated from the police report, called MV104-AN, which is required to be filled out for collisions where someone is injured or killed, or where there is at least $1000 worth of damage.

Data is available from 2012-07-01 onwards, however for this analysis, we will limit ourselves to the period up to 2023-08-15, which is when the data was downloaded.

We find that this data contains over 2 million observations, which allows us to explore several aspects of vehicle crashes in NY's boroughs.

The data dictionary for the data is also available at the URL above.

First, we perform unstructured exploration of the data, and then try to answer the following questions:

- We look for which borough has had the maximum number of crashes reported since 2012.

- We relate the number of crashes to to the borough's population to find out which borough has the maximum number of crashes for every 100,000 people. Even though the data does not have this information, we can combine the crash data with the population estimates for the boroughs also available from the City of New York's website (https://data.cityofnewyork.us/City-Government/New-York-City-Population-by-Borough-1950-2040/xywu-7bv9)

| Borough | Population |
| --- | --- |
| Bronx | 1446788 |
| Brooklyn | 2648452 |
| Manhattan | 1638281 |
| Queens | 2330295 |
| Staten Island | 487155 |  

  
- We look for the leading cause of crashes

- We also look for the top-3 causes of crashes, and try to calculate what proportion of all crashes are caused by these top-3 causes.

- We then look to some of the more serious implications of crashes by examining how many accidents involved at least one fatality.

- We then compute, on average, out of every 1000 accidents, how many have resulted in at least one person dead.

- We also look for missing data and try to compute the proportion of accidents in the data that do not have a Borough code.

- The fields 'VEHICLE TYPE CODE 1' and 'VEHICLE TYPE CODE 2' represent the first two vehicles involved in the accident.  We look for which combinations of vehicles have the most number of accidents.


In [149]:
import pandas as pd
import numpy as np
import seaborn as sns
import os
import matplotlib.pyplot as plt
import phik
from IPython.display import Markdown as md

In [150]:
df = pd.read_pickle("shared/Motor_Vehicle_Collisions_-_Crashes.pkl")
df

,CRASH DATE_CRASH TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET NAME,CROSS STREET NAME,OFF STREET NAME,NUMBER OF PERSONS INJURED,...,CONTRIBUTING FACTOR VEHICLE 2,CONTRIBUTING FACTOR VEHICLE 3,CONTRIBUTING FACTOR VEHICLE 4,CONTRIBUTING FACTOR VEHICLE 5,COLLISION_ID,VEHICLE TYPE CODE 1,VEHICLE TYPE CODE 2,VEHICLE TYPE CODE 3,VEHICLE TYPE CODE 4,VEHICLE TYPE CODE 5
0,2021-09-11 02:39:00,NaN,NaN,NaN,NaN,NaN,WHITESTONE EXPRESSWAY,20 AVENUE,NaN,2.0,...,Unspecified,NaN,NaN,NaN,4455765,Sedan,Sedan,NaN,NaN,NaN
1,2022-03-26 11:45:00,NaN,NaN,NaN,NaN,NaN,QUEENSBORO BRIDGE UPPER,NaN,NaN,1.0,...,NaN,NaN,NaN,NaN,4513547,Sedan,NaN,NaN,NaN,NaN
2,2022-06-29 06:55:00,NaN,NaN,NaN,NaN,NaN,THROGS NECK BRIDGE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4541903,Sedan,Pick-up Truck,NaN,NaN,NaN
3,2021-09-11 09:35:00,BROOKLYN,11208.0,40.667202,-73.866500,"(40.667202, -73.8665)",NaN,NaN,1211 LORING AVENUE,0.0,...,NaN,NaN,NaN,NaN,4456314,Sedan,NaN,NaN,NaN,NaN
4,2021-12-14 08:13:00,BROOKLYN,11233.0,40.683304,-73.917274,"(40.683304, -73.917274)",SARATOGA AVENUE,DECATUR STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4486609,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018240,2023-07-03 18:05:00,NaN,NaN,40.866806,-73.931010,"(40.866806, -73.93101)",RIVERSIDE DRIVE,NaN,NaN,0.0,...,Unspecified,NaN,NaN,NaN,4648110,Sedan,Sedan,NaN,NaN,NaN
2018241,2023-07-22 21:39:00,BRONX,10457.0,40.844177,-73.902920,"(40.844177, -73.90292)",EAST 174 STREET,WEBSTER AVENUE,NaN,1.0,...,NaN,NaN,NaN,NaN,4648117,Sedan,NaN,NaN,NaN,NaN
2018242,2023-07-02 17:55:00,MANHATTAN,10006.0,40.711033,-74.014540,"(40.711033, -74.01454)",WEST STREET,LIBERTY STREET,NaN,0.0,...,NaN,NaN,NaN,NaN,4648366,Taxi,NaN,NaN,NaN,NaN
2018243,2023-07-22 13:15:00,QUEENS,11433.0,40.691580,-73.793190,"(40.69158, -73.79319)",110 AVENUE,157 STREET,NaN,1.0,...,Driver Inattention/Distraction,NaN,NaN,NaN,4648129,Station Wagon/Sport Utility Vehicle,E-Bike,NaN,NaN,NaN


***
### Show some exploratory data analysis here
***

## Borough with the maximum number of crashes reported since 2012

In [151]:
df.groupby('BOROUGH').agg('count').iloc[:,0].sort_values(ascending = False)

BOROUGH
BROOKLYN         441026
QUEENS           372457
MANHATTAN        313266
BRONX            205345
STATEN ISLAND     58297
Name: CRASH DATE_CRASH TIME, dtype: int64

## Borough with the maximum number of crashes adjusted for population

In [160]:
df_population = pd.DataFrame({
    'BOROUGH':['BRONX','BROOKLYN','MANHATTAN','QUEENS','STATEN ISLAND'],
    'Population':[1446788,2648452,1638281,2330295,487155]})
df_population

,BOROUGH,Population
0,BRONX,1446788
1,BROOKLYN,2648452
2,MANHATTAN,1638281
3,QUEENS,2330295
4,STATEN ISLAND,487155


In [161]:
df_1 = df_population.merge(df.groupby('BOROUGH').agg('count'),how = 'inner',left_on='BOROUGH',right_index = True)

In [162]:
df_1['CRASHES_PER_100K'] = df_1['CRASH DATE_CRASH TIME']/df_1['Population']*100000

In [163]:
df_1[['BOROUGH','CRASHES_PER_100K']].sort_values(['CRASHES_PER_100K'], ascending = False)

,BOROUGH,CRASHES_PER_100K
2,MANHATTAN,19121.628097
1,BROOKLYN,16652.217975
3,QUEENS,15983.255339
0,BRONX,14193.164444
4,STATEN ISLAND,11966.827806


## Analyzing the leading cause of crashes

In [179]:
df_1 = df.drop(df[df['CONTRIBUTING FACTOR VEHICLE 1'] == 'Unspecified'].index)

In [180]:
df_1['CONTRIBUTING FACTOR VEHICLE 1']

0            Aggressive Driving/Road Rage
1                       Pavement Slippery
2                   Following Too Closely
4                                     NaN
6                     Passing Too Closely
                        ...              
2018238             Following Too Closely
2018239                  Backing Unsafely
2018240                Turning Improperly
2018242    Driver Inattention/Distraction
2018243    Driver Inattention/Distraction
Name: CONTRIBUTING FACTOR VEHICLE 1, Length: 1325509, dtype: object

In [198]:
df_2 = df_1.groupby('CONTRIBUTING FACTOR VEHICLE 1').agg('count').sort_values('CRASH DATE_CRASH TIME',ascending = False)

In [200]:
df_2['PROPOTION_ACCIDENT'] = (df_2['CRASH DATE_CRASH TIME'])/df_2['CRASH DATE_CRASH TIME'].sum()

In [203]:
df_2['PROPOTION_ACCIDENT'].head(1)

CONTRIBUTING FACTOR VEHICLE 1
Driver Inattention/Distraction    0.30418
Name: PROPOTION_ACCIDENT, dtype: float64

## Top 3 causes of crashes 

In [204]:
df_2['PROPOTION_ACCIDENT'].head(3).sum()

0.47598056643578757

## Analyzing fatalities

In [237]:
df_fatality = df.groupby('NUMBER OF PERSONS KILLED').agg('count').drop(0,axis = 0)

In [239]:
df_fatality['CRASH DATE_CRASH TIME'].sum()

2804

## Likelihood of fatal accidents
#### On average, out of every 1000 accidents, how many have resulted in at least one person dead?

In [264]:
result_death_1000 = df_fatality['CRASH DATE_CRASH TIME'].sum()/df['CRASH DATE_CRASH TIME'].count()*1000
result_death_1000

1.3893258747079764

## Missing data
#### What proportion of accidents in the data do not have a Borough code?

In [251]:
df['BOROUGH'].isna().sum()/df['BOROUGH'].isna().count()

0.3110890897784957

## Which combinations of vehicles have the most number of accidents

In [263]:
df.groupby(['VEHICLE TYPE CODE 1','VEHICLE TYPE CODE 2']).agg('count').sort_values('CRASH DATE_CRASH TIME',ascending = False)

CRASH DATE_CRASH TIME  \
VEHICLE TYPE CODE 1                 VEHICLE TYPE CODE 2                                          
Sedan                               Sedan                                               197944   
PASSENGER VEHICLE                   PASSENGER VEHICLE                                   193260   
Station Wagon/Sport Utility Vehicle Station Wagon/Sport Utility Vehicle                 133780   
Sedan                               Station Wagon/Sport Utility Vehicle                 123915   
Station Wagon/Sport Utility Vehicle Sedan                                               123812   
...                                                                                        ...   
Pick-up Truck                       UNKNOWN                                                  1   
                                    UNK L                                                    1   
                                    U HAU                                                    1   
                                    Truck                                                    1   
�MBU                                Taxi                                                     1   

                                                                         BOROUGH  \
VEHICLE TYPE CODE 1                 VEHICLE TYPE CODE 2                            
Sedan                               Sedan                                 123434   
PASSENGER VEHICLE                   PASSENGER VEHICLE                     139729   
Station Wagon/Sport Utility Vehicle Station Wagon/Sport Utility Vehicle    80485   
Sedan                               Station Wagon/Sport Utility Vehicle    75573   
Station Wagon/Sport Utility Vehicle Sedan                                  74227   
...                                                                          ...   
Pick-up Truck                       UNKNOWN                                    0   
                                    UNK L                                      1   
                                    U HAU                                      1   
                                    Truck                                      0   
�MBU                                Taxi                                       1   

                                                                         ZIP CODE  \
VEHICLE TYPE CODE 1                 VEHICLE TYPE CODE 2                             
Sedan                               Sedan                                  123388   
PASSENGER VEHICLE                   PASSENGER VEHICLE                      139718   
Station Wagon/Sport Utility Vehicle Station Wagon/Sport Utility Vehicle     80451   
Sedan                               Station Wagon/Sport Utility Vehicle     75547   
Station Wagon/Sport Utility Vehicle Sedan                                   74206   
...                                                                           ...   
Pick-up Truck                       UNKNOWN                                     0   
                                    UNK L                                       1   
                                    U HAU                                       1   
                                    Truck                                       0   
�MBU                                Taxi                                        1   

                                                                         LATITUDE  \
VEHICLE TYPE CODE 1                 VEHICLE TYPE CODE 2                             
Sedan                               Sedan                                  182018   
PASSENGER VEHICLE                   PASSENGER VEHICLE                      158933   
Station Wagon/Sport Utility Vehicle Station Wagon/Sport Utility Vehicle    121485   
Sedan                               Station Wagon/Sport Utility Vehicle    114718   
Station Wagon/Sport Utility Vehicle Sedan                                  114002   
... 

***  
## Finished